# Machine, Data & Learning

## Project | Genetic Algorithm

Team 30 | Tribrid
-----------
* Nitin Chandak (2019101024)
* Ayush Sharma (2019101004)

In [542]:
# Usefull Imports
import client as server
import numpy as np
import random
import json
import itertools
import os

### Default Initial Overfit Vector
```
[
    0.0, 
    -1.45799022e-12, 
    -2.28980078e-13,  
    4.62010753e-11, 
    -1.75214813e-10, 
    -1.83669770e-15,  
    8.52944060e-16,  
    2.29423303e-05, 
    -2.04721003e-06, 
    -1.59792834e-08,  
    9.98214034e-10
]
```

In [543]:
# Usefull Global Constants & lambda functions

TEAM_ID = 'colthAUIKUTfdh4qWrnHBhJzkyEm8kt4qIue1BKtyvLItfp8Po'

DEFAULT_INITIAL_OVERFIT_VECTOR = [
    0.0, 
    -1.45799022e-12, 
    -2.28980078e-13,  
    4.62010753e-11, 
    -1.75214813e-10, 
    -1.83669770e-15,  
    8.52944060e-16,  
    2.29423303e-05, 
    -2.04721003e-06, 
    -1.59792834e-08,  
    9.98214034e-10
]

POPULATION_SIZE = 10 # intentionaly chose this as we have to store 10 best vectors
GENERATION_LOOP = 10
MIN_GENE_VAL = -10
MAX_GENE_VAL = 10
MUTATION_PROBABILITY = 0.5
MUTATION_FACTOR = lambda : random.uniform(0.9, 1.1)
MATING_POOL_SIZE = 6

OUTPUT_FILE = 'output.txt'

In [544]:
def is_output_file(fileName):
    '''
    This funtion returns False if 
    filename exist  otherwise True
    '''
    return os.path.exists(fileName)

In [545]:
def is_valid_file(filename):
    '''
    This function checks existence of file
    filename and if exist then checks if it
    empty or not.
    '''
    if is_output_file(filename):
        return (os.stat(filename).st_size != 0)
    else:
        return False

In [546]:
def create_default_output_file():
    '''
    This function will create a default
    output file for dumping our 10 best vector
    before any iteration of algorithm when called.
    '''
    answer = []
    for i in range(10):
        ret_vector = DEFAULT_INITIAL_OVERFIT_VECTOR.copy()
        for j in range(len(ret_vector)):
            if random.randint(1,10)<(10*MUTATION_PROBABILITY):
                if ret_vector[j]==0:
                    ret_vector[j]=random.randint(-10,10)
                else:
                    new_gene = ret_vector[j]*MUTATION_FACTOR()
                    if abs(new_gene)<10:
                        ret_vector[j]=new_gene
        answer.append(ret_vector)
    with open(OUTPUT_FILE,'w') as write_file:
        json.dump(answer, write_file)

In [547]:
def read_output_file():
    '''
    This function will read the OUTPUT_FILE
    and return the answer array there which 
    contains 10 best vector obtained so far.
    '''
    with open(OUTPUT_FILE,'r') as write_file:
        answer=json.load(write_file)
    return answer

In [548]:
def dump_best_vectors(answer: list):
    '''
    This function will dump the list of
    10 vectors into the OUTPUT_FILE file.
    And return the updated data.
    '''
    with open(OUTPUT_FILE,'w') as write_file:
        json.dump(answer, write_file)
    return read_output_file()

In [549]:
#@TODO: Within While loop:
#       Generate new Population
#       Get errors, fitness & curr_weight_distribution
#       Update 10 best vector array i.e. make a separate list for it.
#       Keep track of generations of the vecotrs for generation folder.
#       Dump 10 best vectors into output.txt file.

In [550]:
def get_both_err(population):
    '''
    This function utilises the API 
    call provided to us for getting the
    errors on the vectors within the population.
    
    Parameter
    ---------
    population: list of vector of 11-D
    
    Return
    ------
    It returns two list train_err & validation_err
    which are errors for the given poplation's vectors.
    '''
    train_err = [ random.randint(1,300) for i in range(len(population))]
    validation_err = [ random.randint(1,300) for i in range(len(population))]
    
#     train_err = []
#     validation_err = []
#     for individual in population:
#         [te, ve]= server.get_errors(TEAM_ID,individual)
#         train_err.append(te)
#         validation_err.append(ve)
    return train_err, validation_err

In [551]:
def get_cost(train_error, validation_error):
    '''
    This function calculates the cost
    for given list of errors. Lower the
    cost more fit/perfect the vector.
    
    Parameter
    ---------
    Requires two list train_err & validation_err
    which are errors for the given poplation's vectors.
    
    Return
    ------
    Returns the list of fitness for corresponding errors.
    '''
    cost = []
    for i in range(len(train_error)):
        sum_err = train_error[i] + validation_error[i]
        abs_diff_err = abs(train_error[i] - validation_error[i])
        fit = sum_err + 2 * abs_diff_err
        cost.append(fit)
    return cost

In [552]:
def get_initial_population_info():
    '''
    This function will return a stack
    of all the 10 best vector along with
    their Train error, Validation error
    and their cost
    '''
    initial_population = read_output_file()
    train_error, validation_error = get_both_err(initial_population)
    initial_cost = get_cost(train_error, validation_error)
    initial_population_info = np.column_stack((initial_population,train_error, validation_error,initial_cost))
    return initial_population_info

In [553]:
def create_mating_population(parent_population):
    '''
    This function will select top MATING_POOL_SIZE
    individuals with least costs from the parent_population 
    for cross-over.
    '''
    mating_pool = parent_population[:MATING_POOL_SIZE]
    return mating_pool    

In [554]:
def cross_over(p1,p2):
    '''
    This function simply does the cross-over
    on two individual p1,p2 and returns c1,c2
    i.e. crossed-child.
    '''
    crossover_point = random.randint(1, 10)
    c1 = list(p1[:crossover_point]) + list(p2[crossover_point:])
    c2 = list(p2[:crossover_point]) + list(p1[crossover_point:])
    return c1, c2

def simulate_cross_over(mating_population):
    '''
    This function will perform the cross-over
    on mating_population and generate POPULATION_SIZE
    total childs/individual.
    '''
    mating_population = mating_population[:,:-3]
    crossed_population = []
    for i in range(POPULATION_SIZE//2):
        p1 = mating_population[random.randint(0,MATING_POOL_SIZE-1)]
        p2 = mating_population[random.randint(0,MATING_POOL_SIZE-1)]
        c1, c2 = cross_over(p1,p2)
        crossed_population.append(c1)
        crossed_population.append(c2)
    return crossed_population

In [555]:
def mutate_population(crossed_population):
    '''
    This function will mutate the crossed_population
    and returns the mutated population.
    '''
    mutated_population = []
    for i in range(len(crossed_population)):
        curr_population = crossed_population[i]
        for j in range(len(curr_population)):
            if random.randint(1, 10) <= (10*MUTATION_PROBABILITY):
                new_gene = curr_population[j]*MUTATION_FACTOR()
                if abs(new_gene)<=10:
                    curr_population[j]=new_gene
        mutated_population.append(curr_population)
    return mutated_population

In [556]:
def create_next_generation(mutated_population,parent_population_info):
    '''
    This funtion will get the errors & cost for the mutated population
    then sort it in increasing order. The next_generation will be the 
    combination of 3 top parent & (POPULATION_SIZE-3) mutated childs.
    '''
    train_err, validation_err = get_both_err(mutated_population)
    mutated_cost = get_cost(train_err, validation_err)
    mutated_population_info = np.column_stack((mutated_population,train_err, validation_err,mutated_cost))
    mutated_population_info = mutated_population_info[np.argsort(mutated_population_info[:,-1])]
    
    select_parent = parent_population_info[:3]
    select_mutated = mutated_population_info[:(POPULATION_SIZE-3)]
    
    next_generation = np.concatenate((select_parent,select_mutated))
    next_generation = next_generation[np.argsort(next_generation[:,-1])]
    return next_generation

In [557]:
def submit_vector(vector: list):
    '''
    This function has been written for submitting the best
    vector generated so far for intermediate evaluation.
    `vector` is a list of length 14 with last three
    data as train_err, validation_err & cost.
    '''
    print(vector)
    submit_vector = vector[:-3]
    response = server.submit(TEAM_ID,submit_vector.tolist())
    print(response)

In [558]:
def update_best_vector_set(next_generation,best_vector_set):
    '''
    This function will merge next_generation 
    & best_vector_set in increasing order of cost.
    Then return the top 10 vectors from it.
    '''
    new_best_vector_set = np.concatenate((next_generation,best_vector_set))
    k = new_best_vector_set[np.argsort(new_best_vector_set[:,-1])].tolist()
    new_best_vector_set = [k for k,_ in itertools.groupby(k)]
    new_best_vector_set = np.array(new_best_vector_set)
    return new_best_vector_set[:10]

In [559]:
def GA():
    '''
    Main function to be called to run
    the implemented genetic algorithm.
    '''
    if not is_valid_file(OUTPUT_FILE):
        create_default_output_file()
        print('created a new output.txt')
    
    all_generation = []       
    initial_population_info = get_initial_population_info()
    parent_population = initial_population_info[np.argsort(initial_population_info[:,-1])]
    best_vector_set = parent_population
    
    for generation in range(GENERATION_LOOP-1):
        
        mating_population = create_mating_population(parent_population)
        crossed_population = simulate_cross_over(mating_population)
        mutated_population = mutate_population(crossed_population)
        next_generation = create_next_generation(mutated_population,parent_population)
        best_vector_set = update_best_vector_set(next_generation,best_vector_set)
        
        reproduction_info_dict = {
            'parent_population': parent_population[:,:-3],
            'error': parent_population[:,-3:-1],
            'cost': parent_population[:,-1:],
            'mating_population' : mating_population,
            'crossed_population' : np.array(crossed_population),
            'mutated_population' : np.array(mutated_population),
            'next_generation' : next_generation[:,:-3],
            'next_error': next_generation[:,-3:-1],
            'next_cost': next_generation[:,-1:]
        }
        all_generation.append(reproduction_info_dict)
        parent_population = next_generation
    return best_vector_set

In [560]:
best_vector_set = GA()
print(best_vector_set)

created a new output.txt
[[-9.85788281e+00 -1.61129421e-12 -2.15513714e-13  4.73092182e-11
  -1.84099723e-10 -2.36467711e-15  9.06554179e-16  2.32614407e-05
  -1.78678093e-06 -1.56316135e-08  8.11201855e-10  1.90000000e+01
   7.00000000e+00  5.00000000e+01]
 [-9.72424007e+00 -1.70305982e-12 -2.20221129e-13  4.55677031e-11
  -1.84099723e-10 -1.76079261e-15  9.22712135e-16  2.35954979e-05
  -2.21379245e-06 -1.52170610e-08  1.02863632e-09  2.60000000e+01
   5.50000000e+01  1.39000000e+02]
 [-8.98083009e+00 -1.61129421e-12 -2.20221129e-13  4.55677031e-11
  -1.84099723e-10 -1.92151204e-15  1.01072083e-15  2.51712461e-05
  -2.25727505e-06 -1.63212033e-08  1.14218609e-09  7.40000000e+01
   8.50000000e+01  1.81000000e+02]
 [-9.00000000e+00 -1.61129421e-12 -2.20381430e-13  4.62010753e-11
  -1.75214813e-10 -1.74132296e-15  8.13723584e-16  2.29423303e-05
  -2.04721003e-06 -1.57525118e-08  1.06951609e-09  7.20000000e+01
   8.60000000e+01  1.86000000e+02]
 [-9.00000000e+00 -1.61129421e-12 -2.608455

In [541]:
print("Before =>\n",np.array(read_output_file()))
dump_data = best_vector_set[:,:-3]
print(dump_data)
dump_best_vectors(dump_data.tolist())
print("After =>\n",np.array(read_output_file()))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)